In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
DATA_PATH = 'data/casestudy.csv'

In [3]:
df = pd.read_csv(DATA_PATH, index_col=0)
df.shape

(685927, 3)

In [4]:
df.head()

,customer_email,net_revenue,year
0,nhknapwsbx@gmail.com,249.92,2015
1,joiuzbvcpn@gmail.com,87.61,2015
2,ukkjctepxt@gmail.com,168.38,2015
3,gykatilzrt@gmail.com,62.40,2015
4,mmsgsrtxah@gmail.com,43.08,2015


# The queries can ofcourse be written in SQL but was more convient due to the limited time to write something quickly in pandas. Also the db is not that big but SQL would be the goto for this job in a work environment.

In [9]:
# from sqlalchemy import create_engine
# engine = create_engine(
#     'sqlite:///C:\\Kris_Folder\\Projects\\Stout_projects\\case_study_2\\sales.db', echo=False)


# Total Revenue for Current Year

In [5]:
pd.DataFrame(df.groupby(['year'])['net_revenue'].agg('sum'))

# SELECT DISTINCT COUNT(customer_email), year
# FROM sales_db
# GROUP BY year

,net_revenue
year,
2015,29036749.19
2016,25730943.59
2017,31417495.03


# New Customers

In [8]:
new_customers_count_df = pd.DataFrame({'year' : [], 'customers' : []})
customers = []

for year in df['year'].unique():
    df_current_year = df[df['year'] == year]
    
    new_customers = set(df_current_year['customer_email'].unique()) - set(customers)
    new_customers_count_df = pd.concat([new_customers_count_df, pd.DataFrame({'year' : [year], 'customers' : [len(new_customers)]})], axis=0)
    
    customers = new_customers
new_customers_count_df.reset_index(drop=True)

,year,customers
0,2015.0,231294.0
1,2016.0,145062.0
2,2017.0,235190.0


# New Customer Revenue

In [13]:
new_customers_df = pd.DataFrame()
customers = []

for year in df['year'].unique():
    df_current_year = df[df['year'] == year]
    
    new_customers = set(df_current_year['customer_email'].unique()) - set(customers)
    new_customers_df = pd.concat([new_customers_df, df_current_year[df_current_year['customer_email'].isin(new_customers)]])
    
    customers = new_customers
new_customers_df.reset_index(drop=True)

,customer_email,net_revenue,year
0,nhknapwsbx@gmail.com,249.92,2015
1,joiuzbvcpn@gmail.com,87.61,2015
2,ukkjctepxt@gmail.com,168.38,2015
3,gykatilzrt@gmail.com,62.40,2015
4,mmsgsrtxah@gmail.com,43.08,2015
...,...,...,...
611541,qzqttwiftu@gmail.com,184.58,2017
611542,pjodiifjop@gmail.com,133.03,2017
611543,appaplmgko@gmail.com,200.98,2017
611544,wvkpmwsgck@gmail.com,235.35,2017


In [14]:
pd.DataFrame(new_customers_df.groupby("year")["net_revenue"].agg("sum"))

,net_revenue
year,
2015,29036749.19
2016,18245491.01
2017,29540622.77


# Existing Customer Growth

In [13]:
pd.DataFrame(pd.DataFrame(df.groupby(['year'])['net_revenue'].agg('sum'))['net_revenue'] - pd.DataFrame(df.groupby(['year'])['net_revenue'].agg('sum')).shift(1)['net_revenue'])

,net_revenue
year,
2015,NaN
2016,-3305805.60
2017,5686551.44


# Lost Customers

In [40]:
lost_customers_df = pd.DataFrame()
lost_customers_count_df = pd.DataFrame()
customers = set()

for year in df['year'].unique():
    df_current_year = df[df['year'] == year]
    
    # new_customers = set(df_current_year['customer_email'].unique()) - set(customers)
    # lost_customers_df = pd.concat([lost_customers_df, df_current_year[df_current_year['customer_email'].isin(new_customers)]])
    
    returning_customers = df_current_year[df_current_year['customer_email'].isin(customers)]
    lost_customers = set(customers) - set(returning_customers)
    
    lost_customers_df = pd.concat([lost_customers_df, df_current_year[df_current_year['customer_email'].isin(lost_customers)]])
    lost_customers_count_df = pd.concat([lost_customers_count_df, pd.DataFrame({'year': [year], 'lost_customers' : [len(lost_customers)]})])
    
    customers.update(df_current_year['customer_email'].to_list())
lost_customers_count_df.reset_index(drop=True)

,year,lost_customers
0,2015,0
1,2016,231294
2,2017,376356


# Revenue lost from Attrition

In [41]:
pd.DataFrame(lost_customers_df.groupby('year')['net_revenue'].agg('sum'))

,net_revenue
year,
2016,7485452.58
2017,2740887.39


# Existing Customer Revenue Current Year

In [44]:
pd.DataFrame(df.groupby('year')['net_revenue'].agg(sum))

,net_revenue
year,
2015,29036749.19
2016,25730943.59
2017,31417495.03


# Existing Customer Revenue Prior Year

In [45]:
pd.DataFrame(df.groupby('year')['net_revenue'].agg(sum)).shift(1)

,net_revenue
year,
2015,NaN
2016,29036749.19
2017,25730943.59


# Total Customers Current Year

In [46]:
pd.DataFrame(df.groupby('year')['customer_email'].count())

,customer_email
year,
2015,231294
2016,204646
2017,249987


# Total Customers Previous Year

In [47]:
pd.DataFrame(df.groupby('year')['customer_email'].count()).shift(1)

,customer_email
year,
2015,NaN
2016,231294.0
2017,204646.0
